<a href="https://colab.research.google.com/github/amrios80/GroceryStoreDataset/blob/master/code/taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TALLER 2

### Objetivo:
aplicar técnicas de machine learning, las cuales permitan descubrir insights, sugerir accionables al negocio y calcular el valor ganado.

### Contexto del negocio
Apoyo a un Supermercado Inteligente

### Mision
Mediante el uso de modelos de Machine Learning, en conjunto con técnicas de preparación de datos, se espera que usted esté en capacidad de construir el modelo que identifique los productos, y argumente el valor que generará al supermercado los resultados que obtenga.


## 1. Entendimiento y preparación de los datos:

Reporte de entendimiento de datos.

*   Dimensiones del dataset
*   Caracteristicas de las imagenes
*   Indicadores importantes
*   Intergracion de tecnicas de aumento de datos
*   Determinacion de productos y que categorías a emplear


## 1.1. Dimensiones del dataset



In [ ]:
metadata=pd.read_csv('https://raw.githubusercontent.com/amrios80/GroceryStoreDataset/refs/heads/master/compressed/metadata.csv')
#metadata["mode"].value_counts()
#print(metadata["extension"].value_counts())
print("numero de imagenes " + str(metadata[metadata["path"].str.contains(".jpg")].shape[0]) )
print("archivos por modo:" + str( metadata["mode"].value_counts() ))


numero de imagenes 5418
archivos por modo:mode
train    2639
test     2484
val       295
Name: count, dtype: int64


## 1.2 Caracteristicas de las imagenes



In [ ]:
metadata[["size","size_x","size_y","channels"]].describe()

,size,size_x,size_y,channels
count,5418.000000,5418.000000,5418.000000,5418.0
mean,22420.337025,348.792174,357.120709,3.0
std,4131.906988,9.554136,31.224907,0.0
min,9730.000000,348.000000,348.000000,3.0
25%,19756.000000,348.000000,348.000000,3.0
50%,22048.500000,348.000000,348.000000,3.0
75%,24551.750000,348.000000,348.000000,3.0
max,41049.000000,464.000000,464.000000,3.0


## 1.3 Indicadores

In [ ]:
print("cantidad de clases        :"+str(metadata["class_id"].nunique()))
print("cantidad de coarse classes:"+str(metadata["coarse_class_id"].nunique()))
print("Tamaño en disco           :"+str(metadata["size"].sum()/1000000 )+ " MB ")
print("numero de archivos        :"+str(metadata.shape[0]))
print("Size promedio             :"+str(metadata["size"].mean()/1000 )+ " KB")
print("Categorias                : 3   (Fruit, Packages, Vegetables)")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#1. Entendimiento y preparacion de los datos.
#Dataset:

#Imagenes: 348x348 96dpi 24 bit depth

In [ ]:
#!pip install scikit-learn pandas numpy opencv-python


#training
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data_dir = '/kaggle/input/apples/golden-delicious/'
labels_file = os.path.join(data_dir, 'classes.csv')
label_file_columns=["class_name","class_id","coarse_class_name","coarse_class_id","icon_path","product_description_path"]

# Read the labels
labels_df = pd.read_csv(labels_file)
labels_df.columns=label_file_columns
#print(labels_df)

def  load_file(data_dir='', mode="train"):
    train_file=f"{data_dir}{mode}.txt"
    dftrain_file=pd.read_csv(train_file)
    train_file_columns=["filename","class_id", "coarse_class_id"]
    dftrain_file.columns=train_file_columns
    return dftrain_file
#print(dftrain_file)

dftrain_file=load_file(data_dir=data_dir, mode="train")

print('rows:' + str(dftrain_file.shape[0]))


# Function to load images into an array
def load_images(data_dir, df_files, image_size=348 ):
    images = []
    labels = []
    for index, row in df_files.iterrows():
        img_path = os.path.join(data_dir, row['filename'])
        #print(img_path)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (image_size, image_size))  # Resize to a fixed size
        images.append(img)
        labels.append(row['class_id'])
    return np.array(images), np.array(labels)


def preprocess_dataset(data_dir='', mode="train" ):
    dftrain_file = load_file(data_dir, mode)
    images, labels = load_images(data_dir, dftrain_file )
    # Normalize the images
    # This line normalizes the pixel values of the images by dividing them by 255.0. This scales the pixel values from the original range (typically 0-255 for RGB images) to the range 0-1, which is a common normalization technique for neural networks.
    # images = images / 255.0
    # Flatten the images for the classifier
    #This reshapes the images from their original shape
    # (likely (n_samples, height, width, channels)) to a flat 2D array.
    # The new shape is (n_samples, height * width * channels), where:
    # height * width * channels represents the total number of pixels in each image.
    # n_samples is the number of images in the dataset.
    n_samples, height, width, channels = images.shape
    print('before shaping:')
    print(images.shape)
    print( n_samples, height, width, channels )
    images = images.reshape((n_samples, height * width * channels))
    print('after reshaping:')
    print(images.shape)
    return images, labels


def preprocess_dataset_no_flattening(data_dir='', mode="train" ):
    dftrain_file = load_file(data_dir, mode)
    images, labels = load_images(data_dir, dftrain_file, image_size=128 )
    # Normalize the images
    # This line normalizes the pixel values of the images by dividing them by 255.0. This scales the pixel values from the original range (typically 0-255 for RGB images) to the range 0-1, which is a common normalization technique for neural networks.
    images = images / 255.0
    print(images.shape)
    return images, labels

X_train, y_train = preprocess_dataset( data_dir,"train" )
print(X_train)
print(X_train.shape)
print(y_train)

X_test, y_test = preprocess_dataset( data_dir,"test" )


# Encode the labels if necessary
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# labels = le.fit_transform(labels)

# Split the data
#X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.0, random_state=42)
#X_train =images
#y_train =labels

print(X_train.shape)

###############

# Initialize the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')







rows:277
before shaping:
(277, 348, 348, 3)
277 348 348 3
after reshaping:
(277, 363312)
[[ 37  20 147 ...  29 126 160]
 [ 13  61  89 ...  79 107 142]
 [ 26 144 193 ...  37  82 115]
 ...
 [ 22  33  47 ...  43  68 102]
 [ 43  65 171 ... 104 181 243]
 [  0 114 117 ...  40  35 144]]
(277, 363312)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
before shaping:
(275, 348, 348, 3)
275 348 348 3
after reshaping:
(275, 363312)
(277, 363312)
Accuracy: 59.27%


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Initialize the classifier
xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# Train the classifier
xgb_clf.fit(X_train, y_train)

# Make predictions
y_pred = xgb_clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'XGBoost Accuracy: {accuracy * 100:.2f}%')



KeyboardInterrupt



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D


X_train, y_train = preprocess_dataset_no_flattening( data_dir,"train" )
X_test, y_test = preprocess_dataset_no_flattening( data_dir,"test" )

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Assuming 10 classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'CNN Accuracy: {accuracy * 100:.2f}%')


(277, 128, 128, 3)
(275, 128, 128, 3)
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 523ms/step - accuracy: 0.2620 - loss: 4.0262 - val_accuracy: 0.4436 - val_loss: 1.1451
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 471ms/step - accuracy: 0.5777 - loss: 0.9984 - val_accuracy: 0.5964 - val_loss: 0.8401
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 471ms/step - accuracy: 0.7931 - loss: 0.5782 - val_accuracy: 0.5309 - val_loss: 0.8602
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 455ms/step - accuracy: 0.9280 - loss: 0.3139 - val_accuracy: 0.6727 - val_loss: 0.6822
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 468ms/step - accuracy: 0.9908 - loss: 0.1264 - val_accuracy: 0.7273 - val_loss: 0.6469
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 466ms/step - accuracy: 1.0000 - loss: 0.0425 - val_accuracy: 0.6691 - val_loss: 0.7850
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 445ms/step - accuracy: 1.0000 - loss: 0.0164 - val_accuracy: 0.6509 - val_loss: 0.8393
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 453ms/step - accuracy: 1.0000 - loss: 0.0082 - val_accuracy: 0.6582 - val_loss: 0.8317
Epo

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the classifier
rf_clf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(f'Best Parameters: {best_params}')
print(f'Best Accuracy: {best_accuracy * 100:.2f}%')


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}
Best Accuracy: 67.15%
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   2.7s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=   5.6s
[CV] END max_depth=None, min_samples_split=2, n_estimators=300; total time=   7.8s
[CV] END max_depth=None, min_samples_split=5, n_estimators=300; total time=   6.8s
[CV] END max_depth=None, min_samples_split=10, n_estimators=100; total time=   2.3s
[CV] END max_depth=None, min_samples_split=10, n_estimators=300; total time=   6.4s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   2.4s
[CV] END max_depth=10, min_samples_split=2, n_estimators=200; total time=   4.7s
[CV] END max_depth=10, min_samples_split=2, n_estimators=300; total time=   7.7s
[CV] END max_depth=10, min_samples_split=5, n_estimators=200; total time=   4.6s
[CV] END m

In [ ]:
from sklearn.ensemble import VotingClassifier

# Initialize individual classifiers
clf1 = RandomForestClassifier(n_estimators=100, random_state=42)
clf2 = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
clf3 = LogisticRegression(random_state=42)

# Initialize the Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('rf', clf1), ('xgb', clf2), ('lr', clf3)], voting='soft')

# Train the classifier
voting_clf.fit(X_train, y_train)

# Make predictions
y_pred = voting_clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Voting Classifier Accuracy: {accuracy * 100:.2f}%')


NameError: name 'LogisticRegression' is not defined

In [ ]:
5. Feature Engineering
Improving the quality of your features can significantly boost model performance. This includes creating new features, transforming existing ones, and selecting the most relevant features.

6. Data Augmentation
For image data, augmenting your dataset by applying transformations like rotations, flips, and color adjustments can help improve model generalization.